# Numba and Numpy Optimization Demos

A companion notebook to my presentation for the LINCC Frameworks weekly design discussion.
By Maxine West

## Setup

In [ ]:
import numpy as np
import numba
from astropy.coordinates import SkyCoord
import healpy as hp

## Numba

### Working with Numba

Let's work on a simple problem: converting base 10 numbers into base 60.

In [ ]:
# basic python implementation for converting to a base 60 babylonian number.
# @numba.njit(cache=True)
def base_sixty(n):
    q = n
    r = []

    i = 0
    while(q > 0):
        r_i = q % 60
        r.insert(0, r_i)
        q = q // 60
        i += 1

    return r



to make things prettier for us, let's make a dict of cuneiform characters to use, as well as a utlility function to convert a base 60 number into it's babylonian cuneiform representation. it sure was nice of the ancient baylonians to make their base 60 character system be base 10, may we all aspire to that level of backwards compatibility :)

In [ ]:
ones = [
    "",
    "\U00012415",
    "\U00012416",
    "\U00012417",
    "\U00012418",
    "\U00012419",
    "\U0001241A",
    "\U0001241B",
    "\U0001241C",
    "\U0001241D",
]
tens = [
    "",
    "\U0001230B",
    "\U00012399",
    "\U0001230D",
    "\U0001240F",
    "\U00012410",
]
def babylonianify(t):
    s = ""
    j = 0
    while(j < len(t)):
        ten = t[j] // 10
        one = t[j] % 10
        pos = f"{tens[ten]}{ones[one]}"
        s += pos
        j += 1

    return s

In [ ]:
x = base_sixty(100)
x, babylonianify(x)

In [ ]:
arr = []
for i in range(1,10_000_000):
    arr.append(base_sixty(i))

as we can see, while `numba` does speed up the function compared to the interpreted version, the gains aren't amazing. this is due to two reasons:
- the actual loop for iterating over the numbers is handled by python (in the cell above).
- the math is working with a list dynamically, which numba doesn't like as much

let's see if we can refactor this code to work faster with numba.

### Playing nice with Numba
two optimize our base 60 calculations, will
- statically set the size of our result array
- put our inner loop inside the `njit` compiled function

In [ ]:
# @numba.njit(cache=True)
def base_sixty_static_memory(n=10_000_000):
    max_digits = len(base_sixty(n))
    result = np.zeros((n,max_digits))

    for i in range(0, n):
        j = max_digits - 1
        x = i + 1
        while(x > 0 and j >= 0):
            result[i][j] = x % 60
            x = x // 60
            j -= 1
    
    return result

now let's compare by using this code to generate the first 100 Million base 60 numerals, first by using the python interpreter and then by enabling `njit`.

In [ ]:
base_sixty_static_memory(100_000_000)

## Clever Numpy Array Manipulation

let's start with a simple and unnecessary mathematical calculation.

In [ ]:
def do_something(x, y):
    return np.cos(x) * np.sin(y)

In [ ]:
arr = []
for i in range(0, 10_000_000):
    arr.append(do_something(i, 9_999_999 - i))

for people coming from more explicit and sensible programming langauges, the method `do_something` probably seems like it can only handle operations on two single number variables, x and y. normally, for `do_something` to handle arrays we would have to add a loop to iterate over the data. however, due to the magic/horror of python duck typing and the sleight of hand of the numpy team, we can actually shove a numpy array in there and it'll work much faster.

In [ ]:
x = np.arange(0, 10_000_000, 1)
y = np.arange(10_000_000,0, -1)

do_something(x, y)

this even works multidimmensional arrays!

In [ ]:
x_two_d = x.reshape(-1,100_000)
y_two_d = y.reshape(-1,100_000)

do_something(x_two_d, y_two_d)

### Astronomy Specific Example

A lot of people in astronomy think that the `astropy` coordiante and angular distance calculations are slow (and they kinda are). However, I think this is mostly due to an inefficient pattern that a lot of people use.

In [ ]:
rng = np.random.default_rng(24601)

ras = [(360.0 * rng.random()) for _ in range(100_000)]
decs = [(180.0 * rng.random()) - 90.0 for _ in range(100_000)]

In [ ]:
# get the distance of our random points to the origin
origin = SkyCoord(ra=0.0, dec=0.0, unit="deg")

res = []
for i in range(100_000):
    ra = ras[i]
    dec = decs[i]

    sc = SkyCoord(ra=ra, dec=dec, unit="deg")

    res.append(origin.separation(sc).degree)

res

however, astropy actually supports using `numpy` arrays as inputs for many operations!!

In [ ]:
points = SkyCoord(ra=np.array(ras), dec=np.array(decs), unit="deg")

origin.separation(points).degree

as you can see, the array -> single point separation calculation can be handled just fine by astropy and is many orders faster. and as long as the two arrays are the same shape, astropy can work with them.

### Even More Astronomy Fun
let's try this with an even bigger problem: can we find the distances between a set of points $M$ and every point in another set $N$? I won't even try to make the pure python version, as it would be way to slow. instead, let's just manipulate our two sets to get those results really fast. For this, we will compare the 100 thousand randomly generated points against 400 points sampled from a healpix partition.

In [ ]:
bounds = hp.vec2dir(
    hp.boundaries(2**2, 4, step=100, nest=True), lonlat=True
)

# reshape the ra matrix to be a single column, then repeat each column 400 times (the length of the N array)
ra_matrix = np.repeat(np.reshape([ras], (-1, 1)), 400, axis=1)
dec_matrix = np.repeat(np.reshape([decs], (-1, 1)), 400, axis=1)

# repeat the N matrix 100,000 times
bounds_ra_matrix = np.repeat(np.array([bounds[0]]), 100_000, axis=0)
bounds_dec_matrix = np.repeat(np.array([bounds[1]]), 100_000, axis=0)

np.shape(ra_matrix), np.shape(bounds_dec_matrix)

In [ ]:
sc1 = SkyCoord(ra = ra_matrix, dec = dec_matrix, unit="deg")
sc2 = SkyCoord(ra = bounds_ra_matrix, dec = bounds_dec_matrix, unit="deg")

In [ ]:
sc1.separation(sc2).degree